In [11]:
from pathlib import Path
import duckdb
rootpath = Path.cwd().parent
duckdb_destination_path = rootpath / "data/defi_risk_test.duckdb"


In [15]:
sql = """
SELECT * FROM information_schema.tables
"""

conn = duckdb.connect(str(duckdb_destination_path))
df = conn.execute(sql).fetchdf()
display(df)
conn.close()


,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,defi_risk_test,sfrxeth,log,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,defi_risk_test,sfrxeth,log__topics,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,defi_risk_test,sfrxeth,transaction,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,defi_risk_test,sfrxeth,_dlt_loads,BASE TABLE,None,None,None,None,None,YES,NO,None,None
4,defi_risk_test,sfrxeth,_dlt_pipeline_state,BASE TABLE,None,None,None,None,None,YES,NO,None,None
5,defi_risk_test,sfrxeth,_dlt_version,BASE TABLE,None,None,None,None,None,YES,NO,None,None


In [17]:
sql = """
SELECT * FROM sfrxeth.log
"""
conn = duckdb.connect(str(duckdb_destination_path))
df = conn.execute(sql).fetchdf()
display(df)
conn.close()

,address,data,block_number,block_hash,time_stamp,gas_price,gas_used,log_index,transaction_hash,transaction_index,_dlt_load_id,_dlt_id
0,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0x00000000000000000000000000000000000000000000...,0x1075dc7,0x6f971a452a974a94c9d59bf20c1c6ed66bb5c0ff2782...,0x64612d33,0x9c2847d19,0x1287b2,0xa6,0xcac35a9c033ea94fa885060c2fcce68ac17574250cc6...,0x3f,1750823287.1229367,5pGN/mW2N3bATA
1,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0x00000000000000000000000000000000000000000000...,0x1075dc7,0x6f971a452a974a94c9d59bf20c1c6ed66bb5c0ff2782...,0x64612d33,0x9c2847d19,0x1287b2,0xa7,0xcac35a9c033ea94fa885060c2fcce68ac17574250cc6...,0x3f,1750823287.1229367,To+VISkG17k4/w
2,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0x00000000000000000000000000000000000000000000...,0x1075f25,0x0c72e1aa14e2607d062d6acc56e088d299cb9aedea2a...,0x64613def,0xc0ff737d8,0x141d07,0x62,0xf5cc3b8d70eb97313916d62acdb7936b4996d8c8d430...,0x3e,1750823287.1229367,BkjbCAqitf6JqQ
3,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0x00000000000000000000000000000000000000000000...,0x1075f25,0x0c72e1aa14e2607d062d6acc56e088d299cb9aedea2a...,0x64613def,0xc0ff737d8,0x141d07,0x63,0xf5cc3b8d70eb97313916d62acdb7936b4996d8c8d430...,0x3e,1750823287.1229367,K6nKxJvQRLzvsQ
4,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0x00000000000000000000000000000000000000000000...,0x1076eb1,0xc553074b5408478e26f408da8820402f6a01b9072097...,0x6461fbc7,0xbd28e4657,0x11201d,0x157,0x364382c084cfb2c557809367a123de63cf794f144fd5...,0x73,1750823287.1229367,Qa/Rq0rQxXRTjw
...,...,...,...,...,...,...,...,...,...,...,...,...
3337,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0x00000000000000000000000000000000000000000000...,0x152f18b,0xee1443fabd49426b3f9152943bd2b4343c1f9a4434b6...,0x67f3074b,0x9fc5a75a,0x128561,0x215,0x258b2bd928131f14cc0887d76b0483f44851781631eb...,0x46,1750823293.8462198,R8N7+rzmCoV41w
3338,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0x00000000000000000000000000000000000000000000...,0x152f18b,0xee1443fabd49426b3f9152943bd2b4343c1f9a4434b6...,0x67f3074b,0x9fc5a75a,0x128561,0x216,0x258b2bd928131f14cc0887d76b0483f44851781631eb...,0x46,1750823293.8462198,t5/xzhpzcirBnA
3339,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0x00000000000000000000000000000000000000000000...,0x152f37e,0x5488ec310a78f026559705c0c9c4eb96970b4372067a...,0x67f31ec7,0x6a16ab1e,0xb4185,0x72,0x5b658e78c6765fb8ff5a123e982e64182a0685aee606...,0xd,1750823293.8462198,2IhQ3h1HWJ8cIg
3340,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0x00000000000000000000000000000000000000000000...,0x152f37e,0x5488ec310a78f026559705c0c9c4eb96970b4372067a...,0x67f31ec7,0x6a16ab1e,0xb4185,0x73,0x5b658e78c6765fb8ff5a123e982e64182a0685aee606...,0xd,1750823293.8462198,zfnT4BJ91oiPHg


In [ ]:
sql = """
SELECT * FROM sfrxeth.transaction
"""

conn = duckdb.connect(str(duckdb_destination_path))
df = conn.execute(sql).fetchdf()
display(df)
conn.close()

,block_number,time_stamp,hash,nonce,block_hash,transaction_index,from,to,value,gas,...,txreceipt_status,input,contract_address,cumulative_gas_used,gas_used,confirmations,method_id,function_name,_dlt_load_id,_dlt_id
0,17259975,2023-05-15 01:49:23+07:00,0xcac35a9c033ea94fa885060c2fcce68ac17574250cc6...,5634,0x6f971a452a974a94c9d59bf20c1c6ed66bb5c0ff2782...,63,0x7a16ff8270133f063aab6c9977183d9e72835428,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0,1832081,...,1,0x23cfed03000000000000000000000000000000000000...,,6332080,1214386,5519045,0x23cfed03,"create_loan(uint256 _durationOfLoan,uint256 _i...",1750825161.0105774,TSS81jWorVph/w
1,17260325,2023-05-15 03:00:47+07:00,0xf5cc3b8d70eb97313916d62acdb7936b4996d8c8d430...,383,0x0c72e1aa14e2607d062d6acc56e088d299cb9aedea2a...,62,0xd0c096ac82eba8d7a26f96ffc34b4e3bba3a1122,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0,1993187,...,1,0x23cfed03000000000000000000000000000000000000...,,11048627,1318151,5518695,0x23cfed03,"create_loan(uint256 _durationOfLoan,uint256 _i...",1750825161.0105774,qdt8uNES4Dr10A
2,17264305,2023-05-15 16:30:47+07:00,0x364382c084cfb2c557809367a123de63cf794f144fd5...,5648,0xc553074b5408478e26f408da8820402f6a01b9072097...,115,0x7a16ff8270133f063aab6c9977183d9e72835428,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0,1765226,...,1,0xdd171e7c000000000000000000000000000000000000...,,10452958,1122333,5514715,0xdd171e7c,"borrow_more(uint256 collateral,uint256 debt)",1750825161.0105774,NM5hWsFtyfBygA
3,17266711,2023-05-16 00:38:35+07:00,0x0e15f4dd52c51389702d40b2d8e877487a4820379560...,5661,0x7cc6ad6f446ba78afe784887e779948cab393e4fc6f9...,90,0x7a16ff8270133f063aab6c9977183d9e72835428,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0,1070522,...,1,0xdd171e7c000000000000000000000000000000000000...,,13163023,643002,5512309,0xdd171e7c,"borrow_more(uint256 collateral,uint256 debt)",1750825161.0105774,4lkzzjlrr1mZ7Q
4,17267826,2023-05-16 04:25:59+07:00,0xeb0806212d102c3c25fa4c864a3c9d3835d92bc45f7d...,5312,0x32eee2df13b1624784f537fd5dcab27ed5950786db8e...,120,0x5180db0237291a6449dda9ed33ad90a38787621c,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0,1591210,...,1,0x23cfed03000000000000000000000000000000000000...,,13924936,1213212,5511194,0x23cfed03,"create_loan(uint256 _durationOfLoan,uint256 _i...",1750825161.0105774,1gCskRdSoQOdQw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,22014379,2025-03-10 12:27:23+07:00,0xd799bf323c586ad0430e5a50c183443949e81c49deab...,567,0x5e8f229747da0f073c964311760ff14cef3c63a49c1e...,43,0xc9396c19c5ce1cb80be5a2c7f52d1cef88ac866e,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0,1385663,...,1,0x2e4af52a000000000000000000000000000000000000...,,4883799,857204,764641,0x2e4af52a,"remove_collateral(uint256 collateral,bool use_...",1750825163.735497,O4C+BqmC+58UOQ
1696,22014391,2025-03-10 12:29:47+07:00,0xf876c7955685d911f45c6a5be860163fbb6c9b628c74...,570,0xebd0dbf5d317ff6b9dcf0433801c02bb5b3144c114f9...,371,0xc9396c19c5ce1cb80be5a2c7f52d1cef88ac866e,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0,932316,...,1,0x37671f93000000000000000000000000000000000000...,,22984037,582609,764629,0x37671f93,"repay(uint256 _d_debt,address _for,int256 max_...",1750825163.735497,b3R7/BLJH7IksA
1697,22195372,2025-04-04 18:51:47+07:00,0x010720867c29a41a866919ad907579e449c9ed63b475...,213,0x0743195e3a1f7340da90e186ddb14d3bbddec4f93b20...,220,0x02d489cfdf7b406630263ed659c0e0449c6c1c0c,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0,1500000,...,1,0xbcbaf48700000000000000000000000035a74cf36c28...,,16225557,745896,583648,0xbcbaf487,"liquidate(address vaultOwner, uint256 oTokensT...",1750825163.735497,lmWaljbLAEqKeA
1698,22213502,2025-04-07 07:39:35+07:00,0x5b658e78c6765fb8ff5a123e982e64182a0685aee606...,431,0x5488ec310a78f026559705c0c9c4eb96970b4372067a...,13,0xf20317d989d2577bfb7c3487b47d5f93590b76dc,0x8472a9a7632b173c8cf3a86d3afec50c35548e76,0,991078,...,1,0x37671f93000000000000000000000000000000000000...,,4934843,737669,565518,0x37671f93,"repay(uint256 _d_d